✅ STEP 1: Import a Messy Dataset
We’ll simulate a messy CSV — think like a startup log or scraped data:

In [1]:
import pandas as pd

# Simulated messy data
data = {
    'Name': [' Alice ', 'bob', 'Charlie', None, 'BOB', 'Alice'],
    'Age': ['25', 'thirty', '30', '22', None, '25'],
    'JoinDate': ['2022-01-01', 'not available', '2022/03/10', '2022-04-15', '', '2022-01-01'],
    'Score': [90, 85, None, 88, 85, 90]
}

df = pd.DataFrame(data)
print(df)


      Name     Age       JoinDate  Score
0   Alice       25     2022-01-01   90.0
1      bob  thirty  not available   85.0
2  Charlie      30     2022/03/10    NaN
3     None      22     2022-04-15   88.0
4      BOB    None                  85.0
5    Alice      25     2022-01-01   90.0


🔍 What’s wrong with this data?

Problem	Affects
Whitespaces in names (' Alice ')	Text columns
Duplicate names with case difference	Consistency
Wrong age type ('thirty')	Numeric analysis
Missing values (None, '')	Aggregation
Mixed date formats	Time operations
Duplicate rows	Data accuracy

✅ STEP 2: Identify and Fix Missing Values  - Convert to NAN
We’ll deal with:

- None or NaN values
- Empty strings ""
- Invalid values like "not available"

In [ ]:
# Detect missing values (including empty strings as NaN) - ERROR data
df.replace("", pd.NA, inplace=True)
df.replace("not available", pd.NA, inplace=True)
print(df.isna().sum())


Name        1
Age         1
JoinDate    2
Score       1
dtype: int64


🔧 2. Fix Missing Values - Data cleaning(Missing values, data types mismatch)

Column	Fix Plan
Name	Drop rows with missing name
Age	Set invalid entries to NaN, then drop
JoinDate	Fill missing with a placeholder date
Score	Fill with average

In [5]:
# Drop missing names
df = df[df['Name'].notna()]

# Fix Age column (convert to numeric, errors='coerce' makes 'thirty' → NaN)
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df = df[df['Age'].notna()]

# Fix JoinDate - fill missing
df['JoinDate'] = df['JoinDate'].fillna("2000-01-01")
df['JoinDate'] = pd.to_datetime(df['JoinDate'], errors='coerce')

# Fill missing Score with average
df['Score'] = df['Score'].fillna(df['Score'].mean())


✅ STEP 3: Remove Duplicates & Clean Text Data
This step is about:

Removing exact duplicate rows
Standardizing text values (like " Alice " → "alice")
Lowercasing all names or emails for consistency



In [ ]:
#🔁 1. Remove Exact Duplicates
# Remove fully duplicate rows
df = df.drop_duplicates()


In [7]:
# ✨ 2. Standardize Name, Email, etc.
# Strip whitespace and lowercase all names & emails
df['Name'] = df['Name'].str.strip().str.lower()
df['Email'] = df['Email'].str.strip().str.lower()


KeyError: 'Email'

🧩 STEP 4: Fix Data Types & Format Columns
Real-world datasets often have the wrong types. In this step we’ll:

1. Fix Age, JoinDate, and Score columns
2. Convert dates to datetime
3. Round scores or format numbers properly

In [8]:
# ✅ 1. Check Current Data Types
print(df.dtypes)


Name                object
Age                float64
JoinDate    datetime64[ns]
Score              float64
dtype: object


In [9]:
# 🔧 2. Fix Incorrect Types
# Convert “Age” or “Score” to numeric:
df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Score'] = pd.to_numeric(df['Score'], errors='coerce')

# Convert “JoinDate” to datetime:
df['JoinDate'] = pd.to_datetime(df['JoinDate'], errors='coerce')


In [10]:
#🎯 3. Round Score to 2 Decimal Places
df['Score'] = df['Score'].round(2)


🧩 STEP 5: Filter, Add Columns & Map Values 🔍📊 - start
In this step, we’ll simulate real-world data transformation tasks like filtering important rows, creating new columns, and mapping values.

In [11]:
#✅ 1. Filter Employees (e.g. Age > 30 and Score > 75)
filtered_df = df[(df['Age'] > 30) & (df['Score'] > 75)]
print(filtered_df)


Empty DataFrame
Columns: [Name, Age, JoinDate, Score]
Index: []


In [12]:
# ✅ 2. Add a New Column: “Performance”
# Use conditions to assign:

# - "High" if score ≥ 85
# - "Medium" if score between 70–84
# - "Low" if score < 70
def performance_level(score):
    if score >= 85:
        return 'High'
    elif score >= 70:
        return 'Medium'
    else:
        return 'Low'

df['Performance'] = df['Score'].apply(performance_level)


In [14]:
# ✅ 3. Map Values
# Let’s assume you have a “Department” column:
import numpy as np
df['Department'] = np.random.choice(['HR', 'Eng', 'Mkt'], size=len(df))

department_map = {
    'HR': 'Human Resources',
    'Eng': 'Engineering',
    'Mkt': 'Marketing'
}
df['DepartmentFull'] = df['Department'].map(department_map)


In [15]:
# ✅ 4. Add a Boolean Column (Pass/Fail based on Score)
df['Passed'] = df['Score'] >= 60



🧠 STEP 6: Grouping, Aggregation, Pivot Tables
This step is core for reporting and dashboards.

In [16]:
#✅ 1. Group By Department, Get Mean Score
dept_group = df.groupby('Department')['Score'].mean()
print(dept_group)


Department
Eng    90.0
HR     90.0
Name: Score, dtype: float64


In [17]:
#✅ 2. Group By Department & Performance → Count Employees
grouped = df.groupby(['Department', 'Performance']).size().reset_index(name='Count')
print(grouped)


  Department Performance  Count
0        Eng        High      1
1         HR        High      2


In [19]:
#✅ 3. Aggregation: Multiple Stats (mean, max, count)
agg_stats = df.groupby('Department').agg({
    'Age': ['mean', 'max'],
    'Score': ['mean', 'count']
})
print(agg_stats)


             Age       Score      
            mean   max  mean count
Department                        
Eng         25.0  25.0  90.0     1
HR          27.5  30.0  90.0     2


In [20]:
# ✅ 4. Pivot Table
# Quick view of scores by department and performance level:
pivot = df.pivot_table(values='Score', index='Department', columns='Performance', aggfunc='mean')
print(pivot)


Performance  High
Department       
Eng          90.0
HR           90.0


In [21]:
# ✅ 5. Crosstab (For Category Frequencies)
pd.crosstab(df['Department'], df['Passed'])


Passed,True
Department,
Eng,1
HR,2


🧼 STEP 7: Handling Missing Data (Real-World Cleaning)
This step mirrors real-world raw data cleanup.

In [22]:
#✅ 1. Identify Missing Data
print(df.isna().sum())


Name              0
Age               0
JoinDate          1
Score             0
Performance       0
Department        0
DepartmentFull    0
Passed            0
dtype: int64


In [23]:
# ✅ 2. Drop Rows with Missing Names
df = df[df['Name'].notna()]


In [24]:
#✅ 3. Fill Missing Age with Department-wise Mean Age
df['Age'] = df.groupby('Department')['Age'].transform(lambda x: x.fillna(x.mean()))



In [25]:
#✅ 4. Fill Missing Score with Global Mean
df['Score'] = df['Score'].fillna(df['Score'].mean())


In [26]:
#✅ 5. Fill Missing Performance with 'Average'
df['Performance'] = df['Performance'].fillna('Average')


🧱 STEP 8: Sorting, Renaming, Resetting Index
This step focuses on organizing your dataset for better readability and export.

In [27]:
#✅ 1. Sort by Score Descending (Top performers first)
df = df.sort_values(by='Score', ascending=False)


In [28]:
#✅ 2. Rename Column for Clarity
df = df.rename(columns={'Score': 'Final Score'})


In [29]:
#✅ 3. Reset Index (After Sorting)
df = df.reset_index(drop=True)


In [30]:
#✅ 4. Rename Department Values (Optional Mapping)
dept_map = {'AI': 'Artificial Intelligence', 'Web': 'Web Dev', 'DS': 'Data Science'}
df['Department'] = df['Department'].map(dept_map)


In [31]:
#✅ 5. Print Cleaned Table
print(df.head())


      Name   Age   JoinDate  Final Score Performance Department  \
0    alice  25.0 2022-01-01         90.0        High        NaN   
1  charlie  30.0        NaT         90.0        High        NaN   
2    alice  25.0 2022-01-01         90.0        High        NaN   

    DepartmentFull  Passed  
0      Engineering    True  
1  Human Resources    True  
2  Human Resources    True  


🗃️ STEP 9: Save, Export, and Load Workflow
This step makes your DataFrame portable and shareable like a real data engineer or analyst would do in projects.

In [32]:
#✅ 1. Save Cleaned DataFrame to CSV
df.to_csv("cleaned_students.csv", index=False)


In [33]:
#✅ 2. Load It Back (Test the File Works!)
new_df = pd.read_csv("cleaned_students.csv")
print(new_df.head())


      Name   Age    JoinDate  Final Score Performance  Department  \
0    alice  25.0  2022-01-01         90.0        High         NaN   
1  charlie  30.0         NaN         90.0        High         NaN   
2    alice  25.0  2022-01-01         90.0        High         NaN   

    DepartmentFull  Passed  
0      Engineering    True  
1  Human Resources    True  
2  Human Resources    True  


In [37]:
#✅ 3. Optional: Save to Excel
df.to_excel("cleaned_students.xlsx", index=False)


In [38]:
#✅ 4. Load from Excel (Bonus)
excel_df = pd.read_excel("cleaned_students.xlsx")
print(excel_df.head())


      Name  Age   JoinDate  Final Score Performance  Department  \
0    alice   25 2022-01-01           90        High         NaN   
1  charlie   30        NaT           90        High         NaN   
2    alice   25 2022-01-01           90        High         NaN   

    DepartmentFull  Passed  
0      Engineering    True  
1  Human Resources    True  
2  Human Resources    True  
